## 1. 라이브러리 선언 

In [1]:
import pandas as pd
# 데이터 분석을 위하여 판다스를 선언한다.

import numpy as np
# 원하는 컬럼을 조회하기 위한 방법 중  np.where을 사용하기 위하여 numpy를 선언한다

## 2. csv파일 불러오기

In [2]:
selloutData =\
    pd.read_csv("../dataset/kopo_channel_seasonality_new.csv")

## 3. 타입 통합 & 불량데이터 처리

In [3]:
selloutData.info()
selloutData.dtypes
# 데이터프레임의 칼럼별 정보와 타입을 확인한다.

selloutData.QTY = selloutData.QTY.astype(float) 
selloutData = selloutData.astype({"REGIONID":str, "PRODUCT":str,"YEARWEEK":str}) 
selloutData.info()
# 데이터프레임의 칼럼별 타입을 원하는 타입으로 변환후 확인한다.

selloutData.loc[selloutData.QTY <0]
selloutData["QTY_NEW"] =\
    np.where(selloutData.QTY < 0,
        0, 
        selloutData.QTY)
# # 음수인 데이터를 0값으로 처리하고 QTY_NEW칼럼을 생성한다.

selloutData
selloutData.loc[(selloutData.QTY<0) &
               (selloutData.QTY_NEW !=0)].count()
# 재대로 불량데이터를 처리하고 생성하였는지 확인한다.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124658 entries, 0 to 124657
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   REGIONID  124658 non-null  object 
 1   PRODUCT   124658 non-null  object 
 2   YEARWEEK  124658 non-null  int64  
 3   QTY       124658 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124658 entries, 0 to 124657
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   REGIONID  124658 non-null  object 
 1   PRODUCT   124658 non-null  object 
 2   YEARWEEK  124658 non-null  object 
 3   QTY       124658 non-null  float64
dtypes: float64(1), object(3)
memory usage: 3.8+ MB


REGIONID    0
PRODUCT     0
YEARWEEK    0
QTY         0
QTY_NEW     0
dtype: int64

## 4. 데이터 통합

In [4]:
selloutData["YEAR"]=selloutData.YEARWEEK.str[0:4]
selloutData["WEEK"]=selloutData.YEARWEEK.str[-2:]
selloutData.WEEK = selloutData.WEEK.astype(int)
selloutData

refinedSelloutData =selloutData.loc[selloutData.WEEK<53]
# 연도별 주차수가 다른 YEAR(53주)가 관찰되었으므로 제거 후 데이터를 통합한다.

## 5. 대표값 생성

In [5]:
sortKey = ["REGIONID","PRODUCT","YEARWEEK","QTY"]
# 정렬의 기준이 될 컬럼들을 저장
sortedData = refinedSelloutData.sort_values(by =sortKey,
                               ascending = [True, True, True, True])
# 정렬

# 지역/상품/연도별 판매량 산출(GROUP BY 활용)

groupKey = ["REGIONID","PRODUCT","YEAR"]
# 그룹바이를 할 KEY값들을 저장
groupData= sortedData.groupby(by=groupKey)["QTY_NEW"].agg(["mean"]).reset_index()
# QTY_NEW컬럼의 평균값을 만든후 GROUPDATA로 저장
groupData = groupData.rename(columns = {"mean":"QTY_MEAN"})
# 컬럼명을 QTY_MEAN으로 변경
groupData

,REGIONID,PRODUCT,YEAR,QTY_MEAN
0,A00,PRODUCT34,2014,275.961538
1,A00,PRODUCT34,2015,86.634615
2,A00,PRODUCT34,2016,36.576923
3,A00,PRODUCT58,2014,2.673077
4,A00,PRODUCT58,2015,5.711538
...,...,...,...,...
2377,A77,PRODUCT1,2015,3030.019231
2378,A77,PRODUCT1,2016,3375.326923
2379,A77,PRODUCT12,2014,2035.788462
2380,A77,PRODUCT12,2015,3540.980769


## 6. 계절성 지수 계산

In [6]:
joinKey=["REGIONID","PRODUCT","YEAR"]
# 조인을 하기위하여 

mergedData = pd.merge(left=refinedSelloutData,
         right=groupData,
         on=joinKey,
         how="inner")

# 계절성지수 = 실(QTY)/(지역,상품,연도별판매량 )
# 계절성 지수를 만들기 위하여 정제된 데이터와 대표값데이터를 합친다

mergedData["SEASONALITY"]=mergedData.QTY_NEW/mergedData.QTY_MEAN

## 7. 주차별 효과 계산

In [7]:
# 지역/상품/주차별 지수를 산출한다.

In [8]:
groupKey=["REGIONID","PRODUCT","WEEK"]
finalResult=mergedData.groupby(by=groupKey)["SEASONALITY"].agg(["mean"]).reset_index()
finalResult=finalResult.rename(columns={"mean":"SEASONALITY"})

finalResult

,REGIONID,PRODUCT,WEEK,SEASONALITY
0,A00,PRODUCT34,1,1.570782
1,A00,PRODUCT34,2,1.755540
2,A00,PRODUCT34,3,1.319460
3,A00,PRODUCT34,4,1.490298
4,A00,PRODUCT34,5,1.061909
...,...,...,...,...
41283,A77,PRODUCT12,48,1.352712
41284,A77,PRODUCT12,49,1.094083
41285,A77,PRODUCT12,50,1.386116
41286,A77,PRODUCT12,51,1.255192
